# 트리의 앙상블

## 랜덤포레스트

In [66]:
!pip install xgboost
!pip install lightgbm

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
    --------------------------------------- 0.0/1.4 MB 1.3 MB/s eta 0:00:02
   --------------- ------------------------ 0.5/1.4 MB 8.5 MB/s eta 0:00:01
   -------------------------------- ------- 1.2/1.4 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 10.2 MB/s eta 0:00:00


In [80]:
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np

In [12]:
wine = pd.read_csv('https://bit.ly/wine-date')
data=wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [18]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score= True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 0.9973541965122431 0.8905151032797809

0.9973541965122431 0.8905151032797809


In [22]:
rf.fit(train_input, train_target)

# feature_importances_ : 특성 중요도 (Feature Importance). 각 특성(변수)이 모델 예측에 얼마나 중요한지를 평가
print(rf.feature_importances_)
# [0.23167441 0.50039841 0.26792718]

[0.23167441 0.50039841 0.26792718]


In [24]:
# oob_score : Out-Of-Bag (OOB) 점수. -> 모델의 검증 성능을 평가하는 데 사용
# 즉, 모델을 학습할 때 사용되지 않은 샘플을 검증하는 방식
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
# 교차 검증없이 성능 평가 가능
# 훈련 데이터 효율적으로 사용
# 과적합 방지
# 랜덤 포레스트와 같은 부트스트랩 샘플링을 사용하는 모델에서만 사용
# 교차 검증에 비해 불안정하므로 OOB점수와 교차검증을 함께 사용하는 것이 좋음

rf.fit(train_input, train_target)
print(rf.oob_score_)
# 0.8934000384837406

0.8934000384837406


## 그레디언트 부스팅
GradientBoostingClassifier
- 그래디언트 부스팅 알고리즘을 사용하는 분류 모델
- 이 모델은 여러 개의 결정 트리를 순차적으로 학습
- 각 트리가 이전 트리의 오류를 보완하는 방식으로 작동

In [39]:
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score= True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 0.8881086892152563 0.8720430147331015

0.8881086892152563 0.8720430147331015


In [41]:
# n_estimators(트리의 개) : 그래디언트 부스팅에서 학습할 결정 트리의 개수
# 값이 클수록 모델의 학습 과정에서 더 많은 트리가 추가되어 점진적으로 모델이 개선
# 트리의 개수가 많을수록 과적합(overfitting)의 위험도 커짐

# learning_rate(학습률) : 각 트리가 모델에 추가될 때 가중치를 얼마나 크게 반영할지 결정
# 학습률이 높을수록 각 트리가 모델에 더 큰 영향을 미치게 되며, 낮을수록 학습이 점진적으로 이루어짐
# 작은 학습률(예: 0.01) : 더 많은 트리(n_estimators의 값)가 필요 -> 과적합을 방지하고 더 세밀한 학습 가능
# 큰 학습률(예: 0.1 이상) : 트리 수가 적어도 빠르게 수렴 -> 과적합의 위험이 커짐
gb = GradientBoostingClassifier(n_estimators =500, learning_rate = 0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score= True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 0.9464595437171814 0.8780082549788999

0.9464595437171814 0.8780082549788999


In [47]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)
# [0.15882696 0.6799705  0.16120254]

[0.15882696 0.6799705  0.16120254]


## 히스토그램 기반 부스팅
히스토그램 기반 부스팅(Histogram-based Boosting)
- 그래디언트 부스팅(Gradient Boosting) 알고리즘의 효율적인 버전
- 대규모 데이터셋을 빠르게 학습할 수 있도록 설계된 히스토그램을 기반으로 한 최적화 방법을 사용

히스토그램 기반 최적화(Histogram-based Optimization)
- 연속적인 데이터를 구간으로 나누어 효율적으로 학습하는 기법
- 히스토그램은 데이터를 여러 구간(bins)으로 나누어, 각 구간에 속하는 데이터의 빈도나 분포를 계산하는 방식
- 데이터 분포를 효율적으로 요약하고, 그래디언트 부스팅과 같은 알고리즘에서 효율적인 분할(splitting)을 구현

In [52]:
hgb = HistGradientBoostingClassifier(random_state= 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score= True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#0.9321723946453317 0.8801241948619236

0.9321723946453317 0.8801241948619236


In [82]:
hgb.fit(train_input, train_target)

result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)
# [0.08876275 0.23438522 0.08027708]

[0.08876275 0.23438522 0.08027708]


In [74]:
hgb.score(test_input, test_target)
# 0.8723076923076923

0.8723076923076923

### XGBoost

In [68]:
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score= True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#0.9558403027491312 0.8782000074035686

0.9558403027491312 0.8782000074035686


### LightGBM

In [76]:
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score= True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 0.935828414851749 0.8801251203079884

0.935828414851749 0.8801251203079884
